In this notebook I will plot all the longidutinal data for each individual separately

In [2]:
import src.data.breathe_data as br
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px
import src.data.helpers as dh
from datetime import timedelta
import src.o2_fev1_analysis.smooth as smooth
import pandas as pd
import numpy as np

In [3]:
# df = br.build_O2_FEV1_FEF2575_df(meas_file=2, remove_nan=False)
# df = br.load_meas_from_excel("BR_O2_FEV1_FEF2575_PEF_Nan")
df = br.load_meas_from_excel("BR_O2_FEV1_FEF2575_PEF_Nan_conservative_smoothing")
# df1 = br.load_meas_from_excel("BR_O2_FEV1_FEF2575_PEF_Nan_20240508")
# df2 = br.load_meas_from_excel("BR_O2_FEV1_FEF2575_PEF_Nan_20240410")
# df3 = br.load_meas_from_excel('BR_O2_FEV1_20240410')
# df = br.load_meas_from_excel("BR_O2_FEV1_FEF2575_PEF_inferred_AR_IA")

# Look at raw Breathe data

In [3]:
dfraw = pd.read_csv(
    dh.get_path_to_main()
    + f"DataFiles/BR/MeasurementData/Breathe_Spirometer_20231113.csv",
)

In [32]:
print(dfraw.shape)
dfraw.head(1)

(104324, 26)


,UserId,EntityId,Timestamp,ClientTimestamp,IsDeleted,CaptureType,CaptureDeviceMake,CaptureDeviceModel,ThirdPartyId,PEF,...,FEV1DivFVC,FEF2575,FEV1Percent,EVol,FVC,InvalidEntry,VolumeTimeCurve,FlowVolumeCurve,IsCapturePrimary,CaptureSessionId
0,60d96f45-ec56-470b-89ae-0085c9f073a7,1000001,2022-08-04 17:15:31.3786092,2022-07-29 16:19:05.1254750 +00:00,False,Bluetooth,Vitalograph,Lung Monitor BLE,NaN,326.0,...,NaN,2.53,88.0,NaN,NaN,False,NaN,NaN,True,00000000-0000-0000-0000-000000000000


In [39]:
dfraw = dfraw[~dfraw.IsDeleted]
dfraw = dfraw[~dfraw.InvalidEntry]
dfraw = dfraw[dfraw.PEF != 0]
print(dfraw.shape)
dfraw.head(1)

(60177, 26)


,UserId,EntityId,Timestamp,ClientTimestamp,IsDeleted,CaptureType,CaptureDeviceMake,CaptureDeviceModel,ThirdPartyId,PEF,...,FEV1DivFVC,FEF2575,FEV1Percent,EVol,FVC,InvalidEntry,VolumeTimeCurve,FlowVolumeCurve,IsCapturePrimary,CaptureSessionId
0,60d96f45-ec56-470b-89ae-0085c9f073a7,1000001,2022-08-04 17:15:31.3786092,2022-07-29 16:19:05.1254750 +00:00,False,Bluetooth,Vitalograph,Lung Monitor BLE,NaN,326.0,...,NaN,2.53,88.0,NaN,NaN,False,NaN,NaN,True,00000000-0000-0000-0000-000000000000


In [45]:
dfpat = br.load_patient_df_from_excel().drop(
    columns=["Age", "Sex", "Height", "StudyNumber"]
)
dfpat.head()

,ID,PartitionKey
0,101,bf306cf3-e0e3-4293-ae15-c4e8c2f85e34
1,102,6cb8bcf5-013f-47b0-8073-3b139e3fca7f
2,103,b8250d09-f356-410f-b2e6-6f1809760b96
3,104,e28915e3-e767-496b-b377-97b075891d48
4,105,2cdf6f89-78d4-439a-b7ab-ff18e191420b


In [48]:
dfraw = dfraw.merge(dfpat, right_on="PartitionKey", left_on="UserId")

In [57]:
print(sorted(dfraw[dfraw.PEF == 300].ID.unique()))

fig = px.histogram(dfraw, x="PEF", nbins=500, color="ID")
# fig = px.histogram(df1, x="FEF2575 % Predicted", nbins=500)
fig.update_layout(width=1500, height=700)
fig.show()

['104', '106', '107', '112', '116', '117', '122', '158', '162', '172', '176', '202', '203', '209', '220', '221', '224', '229', '230', '238', '239', '240', '252', '265', '272', '291', '292', '310', '319', '334', '339']


AttributeError: 'DataFrame' object has no attribute 'ID'

# Profile

In [4]:
def plot_profile_for_ID_raw(df_for_ID):
    df_for_ID = df_for_ID.reset_index()
    fig = make_subplots(
        rows=4,
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.04,
    )
    fig.add_trace(
        go.Scatter(
            y=df_for_ID["O2 Saturation"], x=df_for_ID["Date Recorded"], mode="markers"
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(y=df_for_ID.FEV1, x=df_for_ID["Date Recorded"], mode="markers"),
        row=2,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            y=df_for_ID.FEF2575,
            x=df_for_ID["Date Recorded"],
            mode="markers",
        ),
        row=3,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            y=df_for_ID.PEF,
            x=df_for_ID["Date Recorded"],
            mode="markers",
        ),
        row=4,
        col=1,
    )
    fig.update_yaxes(
        title="SpO2<br>(%)",
        row=1,
        col=1,
        range=[
            np.nanmin(df["O2 Saturation"]) * 0.98,
            np.nanmax(df["O2 Saturation"]) * 1.02,
        ],
    )
    fig.update_yaxes(
        title="FEV1<br>(L)",
        row=2,
        col=1,
        range=[np.nanmin(df.FEV1) * 0.98, np.nanmax(df.FEV1) * 1.02],
    )
    fig.update_yaxes(
        title="FEF2575<br>(L/s)",
        row=3,
        col=1,
        range=[np.nanmin(df.FEF2575) * 0.98, np.nanmax(df.FEF2575) * 1.02],
    )
    fig.update_yaxes(
        title="PEF<br>(L/min)",
        row=4,
        col=1,
        range=[np.nanmin(df.PEF) * 0.98, np.nanmax(df.PEF) * 1.02],
    )
    fig.update_yaxes(nticks=10)
    fig.update_traces(marker=dict(size=3))
    title = f"Measures for ID {df_for_ID.ID[0]}"
    fig.update_layout(
        font=dict(size=8), showlegend=False, title=title, height=1000, width=1400
    )
    # fig.show()
    fig.write_image(f"{dh.get_path_to_main()}PlotsBreathe/ID_profiles/{title}.pdf")
    return


df[df.ID == "405"].groupby(by="ID").apply(plot_profile_for_ID_raw)

""


### List IDs with PEF consistency issues
I will send those IDs to Magic Bullet for them to investigate on the issue's cause: device issue or data manipulation issue?

In [68]:
# ['104' '106' '107' '112' '116' '122' '158' '162' '176' '202' '220' '221'
#  '224' '229' '238' '265' '272' '310' '334']

for id in [
    "104",
    "106",
    "107",
    "112",
    "116",
    "122",
    "158",
    "162",
    "176",
    "202",
    "220",
    "221",
    "224",
    "229",
    "238",
    "265",
    "272",
    "310",
    "334",
]:
    print(id)
    dftmp = df[df.ID == id].dropna(subset="PEF")
    # dftmp = df[df.ID == '104'].dropna(subset='PEF')
    # dftmp = df[df.ID == '334'].dropna(subset='PEF')
    # dftmp = dftmp[dftmp.PEF < 500]
    fig = px.scatter(dftmp, x="Date Recorded", y="PEF")
    fig.update_traces(marker=dict(size=4))
    # fig.show()
    fig.write_image(
        dh.get_path_to_main() + f"PlotsBreathe/IDs with gap in PEF/{id}.pdf"
    )

# 1. ID that show this perfectly 104, 176, 202
# 2. ID that show it well 122, 224, 272, 334
# 3. ID that have huge issue: 238

104
106
107
112
116
122
158
162
176
202
220
221
224
229
238
265
272
310
334


# Profile with smoothing

In [5]:
df1 = br.add_drug_therapy_to_df(df)
drug_df = br.load_drug_therapies()

ERROR:root:ID 175 - ?? Symkevi start date is wrong, removing it because no clue about the true date (maybe 2019?)
ERROR:root:ID 206 - Updating Ivacaftor stop date to not overlap and changing Ivacaftor + Symkevi to Trikafta
ERROR:root:Overlap in drug therapy for ID 405; previous drug therapy not ended
ERROR:root:Overlap in drug therapy for ID 420; stop date > start date
ERROR:root:Overlap in drug therapy for ID 464; previous drug therapy not ended
ERROR:root:ID 175 - ?? Symkevi start date is wrong, removing it because no clue about the true date (maybe 2019?)
ERROR:root:ID 206 - Updating Ivacaftor stop date to not overlap and changing Ivacaftor + Symkevi to Trikafta
ERROR:root:Overlap in drug therapy for ID 405; previous drug therapy not ended
ERROR:root:Overlap in drug therapy for ID 420; stop date > start date
ERROR:root:Overlap in drug therapy for ID 464; previous drug therapy not ended


In [7]:
df1["ecFEF2575%ecFEV1"] = df1["ecFEF2575"] / df1["ecFEV1"] * 100
df1.head()

,ID,Date Recorded,FEV1,O2 Saturation,FEF2575,PEF,PEF (L/s),ecFEV1,ecFEF2575,ecPEF (L/s),Sex,Height,Age,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy,DrugTherapyType,ecFEF2575%ecFEV1
0,101,2019-01-25,1.31,97.0,0.54,NaN,NaN,1.31,0.54,NaN,Male,173.0,53,3.610061,97.150104,36.287474,36.287474,99.845492,None,41.221374
1,101,2019-01-26,1.31,98.0,0.57,NaN,NaN,1.31,0.57,NaN,Male,173.0,53,3.610061,97.150104,36.287474,36.287474,100.874827,None,43.511450
2,101,2019-01-27,1.31,96.0,0.67,NaN,NaN,1.31,0.67,NaN,Male,173.0,53,3.610061,97.150104,36.287474,36.287474,98.816157,None,51.145038
3,101,2019-01-28,1.30,96.0,0.69,NaN,NaN,1.30,0.69,NaN,Male,173.0,53,3.610061,97.150104,36.010470,36.010470,98.816157,None,53.076923
4,101,2019-01-29,1.28,98.0,0.60,NaN,NaN,1.28,0.60,NaN,Male,173.0,53,3.610061,97.150104,35.456463,35.456463,100.874827,None,46.875000


In [9]:
def drug_therapy_color_dict():
    return {
        "Trikafta": "green",
        "Ivacaftor": "purple",
        "Symkevi": "purple",
        "Orkambi": "purple",
    }


def add_drug_therapy_shapes_for_ID(fig, df_for_ID, drug_df):
# def add_drug_therapy_shapes_for_ID(fig, df_for_ID, drug_df):
    drug_df = drug_df[drug_df["DrugTherapyType"] != "Unknown"]
    drug_df_for_ID = drug_df[drug_df.ID == df_for_ID.ID[0]]
    shapes = []
    for _, row in drug_df_for_ID.iterrows():
        start_date = row.DrugTherapyStartDate
        stop_date = row.DrugTherapyStopDate
        if pd.isnull(stop_date):
            stop_date = df["Date Recorded"].max()

        for yref in ["y1", "y2", "y3", "y4", "y5"]:
            fig.add_shape(
                dict(
                    type="rect",
                    # xref="x",
                    yref=yref,
                    # yref="paper",
                    x0=start_date,
                    y0=0,
                    x1=stop_date,
                    y1=1000,
                    fillcolor=drug_therapy_color_dict()[row.DrugTherapyType],
                    opacity=0.08,
                    layer="below",
                    line_width=0,
                    name=row.DrugTherapyType,
                    # label=dict(text=row.DrugTherapyType, textposition="top center", font=dict(size=20)),
                )
            )
        # Add annotation
        fig.add_annotation(
            x=start_date,
            y=1.02,
            xref="x",
            yref="paper",
            # yref="y",
            text=row.DrugTherapyType,
            showarrow=False,
            font=dict(size=10),
        )
    return shapes


def plot_profile_for_ID(df_for_ID):
    df_for_ID = df_for_ID.reset_index()

    fig = make_subplots(
        rows=5,
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.01,
    )

    fig.add_trace(
        go.Scatter(
            y=df_for_ID["O2 Saturation"], x=df_for_ID["Date Recorded"], mode="markers"
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(y=df_for_ID.FEV1, x=df_for_ID["Date Recorded"], mode="markers"),
        row=2,
        col=1,
    )
    fig.update_traces(marker=dict(color="yellow"), row=2, col=1)
    fig.add_trace(
        go.Scatter(y=df_for_ID.ecFEV1, x=df_for_ID["Date Recorded"], mode="markers"),
        row=2,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            y=df_for_ID.FEF2575,
            x=df_for_ID["Date Recorded"],
            mode="markers",
        ),
        row=3,
        col=1,
    )
    fig.update_traces(marker=dict(color="yellow"), row=3, col=1)
    fig.add_trace(
        go.Scatter(y=df_for_ID.ecFEF2575, x=df_for_ID["Date Recorded"], mode="markers"),
        row=3,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            y=df_for_ID["ecFEF2575%ecFEV1"],
            x=df_for_ID["Date Recorded"],
            mode="markers",
        ),
        row=4,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            y=df_for_ID["PEF (L/s)"],
            x=df_for_ID["Date Recorded"],
            mode="markers",
        ),
        row=5,
        col=1,
    )
    fig.update_traces(marker=dict(color="yellow"), row=5, col=1)
    fig.add_trace(
        go.Scatter(
            y=df_for_ID["ecPEF (L/s)"], x=df_for_ID["Date Recorded"], mode="markers"
        ),
        row=5,
        col=1,
    )
    fig.update_yaxes(
        title="SpO2<br>(%)",
        row=1,
        col=1,
        range=[
            np.nanmin(df["O2 Saturation"]) * 0.98,
            np.nanmax(df["O2 Saturation"]) * 1.02,
        ],
    )
    fig.update_yaxes(
        title="FEV1<br>(L)",
        row=2,
        col=1,
        range=[np.nanmin(df.FEV1) * 0.98, np.nanmax(df.FEV1) * 1.02],
    )
    fig.update_yaxes(
        title="FEF2575<br>(L/s)",
        row=3,
        col=1,
        range=[np.nanmin(df.FEF2575) * 0.98, np.nanmax(df.FEF2575) * 1.02],
    )
    fig.update_yaxes(
        title="ecFEF2575%ecFEV1",
        row=4,
        col=1,
        range=[
            np.nanmin(df["ecFEF2575%ecFEV1"]) * 0.98,
            max(200, np.nanmax(df_for_ID["ecFEF2575%ecFEV1"])) * 1.02,
        ],
    )
    fig.update_yaxes(
        title="PEF<br>(L/s)",
        row=5,
        col=1,
        range=[np.nanmin(df.PEF) / 60 * 0.98, np.nanmax(df.PEF) / 60 * 1.02],
    )
    fig.update_yaxes(nticks=10)
    fig.update_traces(marker=dict(size=4))
    title = f"Measures for ID {df_for_ID.ID[0]}, age {df_for_ID.Age[0]}y, height {df_for_ID.Height[0]}cm, {df_for_ID.Sex[0]}"
    fig.update_layout(
        font=dict(size=8),
        showlegend=False,
        title=title,
        height=1000,
        width=1400,
    )
    add_drug_therapy_shapes_for_ID(fig, df_for_ID, drug_df),
    # fig.show()
    fig.write_image(
        f"{dh.get_path_to_main()}PlotsBreathe/ID_profiles_with_conservative_smoothing/{title}.pdf"
    )
    return


# df1[df1.ID == "101"].groupby(by="ID").apply(plot_profile_for_ID)
df1.groupby(by="ID").apply(plot_profile_for_ID)

/var/folders/zq/v2r6yn111s3gpdf8lzf72xvw0000gn/T/ipykernel_66223/260413788.py:150: RuntimeWarning:

All-NaN axis encountered

/var/folders/zq/v2r6yn111s3gpdf8lzf72xvw0000gn/T/ipykernel_66223/260413788.py:150: RuntimeWarning:

All-NaN axis encountered

/var/folders/zq/v2r6yn111s3gpdf8lzf72xvw0000gn/T/ipykernel_66223/260413788.py:150: RuntimeWarning:

All-NaN axis encountered

/var/folders/zq/v2r6yn111s3gpdf8lzf72xvw0000gn/T/ipykernel_66223/260413788.py:150: RuntimeWarning:

All-NaN axis encountered

/var/folders/zq/v2r6yn111s3gpdf8lzf72xvw0000gn/T/ipykernel_66223/260413788.py:150: RuntimeWarning:

All-NaN axis encountered

/var/folders/zq/v2r6yn111s3gpdf8lzf72xvw0000gn/T/ipykernel_66223/260413788.py:150: RuntimeWarning:

All-NaN axis encountered

/var/folders/zq/v2r6yn111s3gpdf8lzf72xvw0000gn/T/ipykernel_66223/260413788.py:150: RuntimeWarning:

All-NaN axis encountered

/var/folders/zq/v2r6yn111s3gpdf8lzf72xvw0000gn/T/ipykernel_66223/260413788.py:150: RuntimeWarning:

All-NaN axis encou

KeyError: nan

In [5]:
df1[df1.FEV1 > 2.3]

,ID,Date Recorded,FEV1,O2 Saturation,FEF2575,PEF,ecFEV1,ecFEF2575,ecPEF,Age,...,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy,PEF (L/s),ecFEV1_2,ecFEV1_3,ecFEF2575_2,ecFEF2575_3,ecPEF (L/s)_2,ecPEF (L/s)_3
337,104,2023-10-29,2.74,97.0,NaN,NaN,2.74,NaN,NaN,25,...,109.591681,109.591681,98.467995,NaN,2.74,2.74,NaN,NaN,NaN,NaN
